In [7]:
import numpy as np
import cv2
import os
from imutils import contours

In [10]:
def cv_show(name, img):
    cv2.imshow(name, img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


def resize(image, width=None, height=None, inter=cv2.INTER_AREA):
    dim = None
    (h, w) = image.shape[:2]
    if width is None and height is None:
        return image
    if width is None:
        r = height / float(h)
        dim = (int(w * r), height)
    else:
        r = width / float(w)
        dim = (width, int(h * r))
    resized = cv2.resize(image, dim, interpolation=inter)
    return resized


def load_digits():
    # 加载数字模板
    path = numbers_address
    filename = os.listdir(path)
    for file in filename:
        # print(file)
        img = cv2.imread(
            numbers_address + "\\" + file)
        img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        img_temp = cv2.threshold(
            img_gray, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
        cnt = cv2.findContours(img_temp, cv2.RETR_EXTERNAL,
                               cv2.CHAIN_APPROX_NONE)[0]
        x, y, w, h = cv2.boundingRect(cnt[0])
        digit_roi = cv2.resize(img_temp[y:y+h, x:x+w], (57, 88))
        # 将数字模板存到列表中
        digits.append(digit_roi)


def demo(index):
    rectKernel = cv2.getStructuringElement(cv2.MORPH_RECT, (25, 25))
    sqKernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))
    target_path = now_dir + "\\" + "demo_" + str(index) + ".png"
    img_origin = cv2.imread(target_path)
    img_origin = resize(img_origin, width=300)
    img_gray = cv2.cvtColor(img_origin, cv2.COLOR_BGR2GRAY)
    gaussian = cv2.GaussianBlur(img_gray, (5, 5), 1)
    img_temp = cv2.threshold(
        gaussian, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
    img_top = cv2.morphologyEx(img_temp, cv2.MORPH_TOPHAT, rectKernel)
    img_sobel_x = cv2.Sobel(img_top, cv2.CV_64F, 1, 0, ksize=7)
    img_sobel_x = cv2.convertScaleAbs(img_sobel_x)
    img_sobel_y = cv2.Sobel(img_top, cv2.CV_64F, 0, 1, ksize=7)
    img_sobel_y = cv2.convertScaleAbs(img_sobel_y)
    img_sobel_xy = cv2.addWeighted(img_sobel_x, 1, img_sobel_y, 1, 0)
    img_closed = cv2.morphologyEx(img_sobel_xy, cv2.MORPH_CLOSE, rectKernel)
    thresh = cv2.threshold(
        img_closed, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
    img_closed = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, sqKernel)
    cnts = cv2.findContours(
        img_closed.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)[0]
    (cnts, boundingBoxes) = contours.sort_contours(cnts, "top-to-bottom")
    draw_img = img_origin.copy()
    draw_img = cv2.drawContours(draw_img, cnts, -1, (0, 0, 255), 1)
    cv_show("666", draw_img)

    # 存放正确数字序列（包含逗号）的轮廓，即过滤掉不需要的轮廓
    right_loc = []
    for c in cnts:
        x, y, w, h = cv2.boundingRect(c)
        ar = w/float(h)
        if ar > 2:
            right_loc.append((x, y, w, h))
    for (gx, gy, gw, gh) in right_loc:
        # 用于存放识别到的数字
        digit_out = []
        if (gy-10 < 0):
            now_gy = gy
        else:
            now_gy = gy-10
        if (gx - 10 < 0):
            now_gx = gx
        else:
            now_gx = gx-10
        img_digit = gaussian[now_gy:gy+gh+10, now_gx:gx+gw+10]
        # 二值化处理
        img_thresh = cv2.threshold(
            img_digit, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
        # 寻找轮廓 找出每个数字的轮廓（包含逗号） 正确的话应该有9个轮廓
        digitCnts = cv2.findContours(
            img_thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)[0]
        # 从左到右排列
        (cnts, boundingBoxes) = contours.sort_contours(digitCnts, "left-to-right")
        cnts = list(cnts)
        flag = 0
        if len(cnts) == 9:
            del cnts[1]
            del cnts[2]
            del cnts[3]
            del cnts[4]
            cnts = tuple(cnts)
            num_roi = []
            for c in cnts:
                x, y, w, h = cv2.boundingRect(c)
                num_roi.append((x, y, w, h))
            for (rx, ry, rw, rh) in num_roi:
                roi = img_digit[ry:ry+rh, rx:rx+rw]
                roi = cv2.resize(roi, (57, 88))
                roi = cv2.GaussianBlur(roi, (5, 5), 1)
                roi = cv2.threshold(
                    roi, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
                source = []
                for digitROI in digits:
                    res = cv2.matchTemplate(
                        roi, digitROI, cv2.TM_CCOEFF_NORMED)
                    max_val = cv2.minMaxLoc(res)[1]
                    source.append(max_val)
                digit_out.append(str(source.index(max(source))))
            cv2.rectangle(img_origin, (gx-5, gy-5),
                          (gx+gw+5, gy+gh+5), (0, 0, 255), 1)
            print(digit_out)
        else:
            print("读取失败")
            flag = 1
        t = ''
        with open(now_dir + "\\temp.txt", 'a+') as q:
            if flag == 0:
                for content in digit_out:
                    t = t + str(content) + " "
                q.write(t.strip(" "))
                q.write('\n')
                t = ''
            else:
                q.write("读取失败")
                q.write('\n')

In [11]:
# 存放数字模板列表
digits = []
# 当前运行目录
now_dir = os.getcwd()
print("当前运行目录：" + now_dir)
numbers_address = now_dir + "\\numbers"
load_digits()
times = input("请输入程序运行次数：")
for i in range(1, int(times) + 1):
    demo(i)
print("输出成功，请检查本地temp.txt文件")
cv2.waitKey(0)
cv2.destroyAllWindows()
while True:
    if input("输入小写‘q’并回车退出") == 'q':
        break

当前运行目录：F:\GitR\code practice\optical_attempt
请输入程序运行次数：1
['0', '1', '7', '2', '3']
['1', '6', '6', '6', '7']
['7', '6', '3', '4', '4']
['9', '2', '9', '5', '3']
['1', '1', '2', '7', '0']
['6', '5', '1', '7', '0']
['8', '2', '4', '9', '6']
['2', '0', '9', '2', '1']
['8', '3', '4', '8', '4']
['2', '8', '3', '5', '7']
['7', '2', '4', '2', '5']
['4', '9', '4', '3', '3']
['5', '9', '1', '3', '1']
['2', '0', '5', '1', '3']
['2', '3', '8', '3', '8']
['7', '0', '6', '8', '8']
输出成功，请检查本地temp.txt文件
输入小写‘q’并回车退出q


In [18]:
# coding=utf-8
# 导入一些python包
from imutils.perspective import four_point_transform
from imutils import contours
import imutils
import cv2
 
# 定义每一个数字对应的字段
DIGITS_LOOKUP = {
    (1, 1, 1, 0, 1, 1, 1): 0,
    (0, 0, 1, 0, 0, 1, 0): 1,
    (1, 0, 1, 1, 1, 1, 0): 2,
    (1, 0, 1, 1, 0, 1, 1): 3,
    (0, 1, 1, 1, 0, 1, 0): 4,
    (1, 1, 0, 1, 0, 1, 1): 5,
    (1, 1, 0, 1, 1, 1, 1): 6,
    (1, 0, 1, 0, 0, 1, 0): 7,
    (1, 1, 1, 1, 1, 1, 1): 8,
    (1, 1, 1, 1, 0, 1, 1): 9
}
 
# 读取输入图片
image = cv2.imread("example.jpg")
 
# 将输入图片裁剪到固定大小
image = imutils.resize(image, height=500)
# 将输入转换为灰度图片
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
# 进行高斯模糊操作
blurred = cv2.GaussianBlur(gray, (5, 5), 0)
# 执行边缘检测
edged = cv2.Canny(blurred, 50, 200, 255)
cv2.imwrite('edge.png', edged)
 
# 在边缘检测map中发现轮廓
cnts = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cnts = imutils.grab_contours(cnts)
# 根据大小对这些轮廓进行排序
cnts = sorted(cnts, key=cv2.contourArea, reverse=True)
displayCnt = None
 
# 循环遍历所有的轮廓
for c in cnts:
    # 对轮廓进行近似
    peri = cv2.arcLength(c, True)
    approx = cv2.approxPolyDP(c, 0.02 * peri, True)
 
    # 如果当前的轮廓有4个顶点，我们返回这个结果，即LCD所在的位置
    if len(approx) == 4:
        displayCnt = approx
        break
 
# 应用视角变换到LCD屏幕上
warped = four_point_transform(gray, displayCnt.reshape(4, 2))
cv2.imwrite('warped.png', warped)
output = four_point_transform(image, displayCnt.reshape(4, 2))
 
# 使用阈值进行二值化
thresh = cv2.threshold(warped, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
cv2.imwrite('thresh1.png', thresh)
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (1, 5))
# 使用形态学操作进行处理
thresh = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel)
cv2.imwrite('thresh2.png', thresh)
 
# 在阈值图像中查找轮廓，然后初始化数字轮廓列表
cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cnts = imutils.grab_contours(cnts)
digitCnts = []
 
# 循环遍历所有的候选区域
for c in cnts:
    # 计算轮廓的边界框
    (x, y, w, h) = cv2.boundingRect(c)
 
    # 如果当前的这个轮廓区域足够大，它一定是一个数字区域
    if w >= 15 and (h >= 30 and h <= 40):
        digitCnts.append(c)
 
# 从左到右对这些轮廓进行排序
digitCnts = contours.sort_contours(digitCnts, method="left-to-right")[0]
digits = []
 
# 循环处理每一个数字
i = 0
for c in digitCnts:
    # 获取ROI区域
    (x, y, w, h) = cv2.boundingRect(c)
    roi = thresh[y:y + h, x:x + w]
 
    # 分别计算每一段的宽度和高度
    (roiH, roiW) = roi.shape
    (dW, dH) = (int(roiW * 0.25), int(roiH * 0.15))
    dHC = int(roiH * 0.05)
 
    # 定义一个7段数码管的集合
    segments = [
        ((0, 0), (w, dH)),               # 上
        ((0, 0), (dW, h // 2)),           # 左上
        ((w - dW, 0), (w, h // 2)),          # 右上
        ((0, (h // 2) - dHC) , (w, (h // 2) + dHC)), # 中间
        ((0, h // 2), (dW, h)),            # 左下
        ((w - dW, h // 2), (w, h)),          # 右下
        ((0, h - dH), (w, h))              # 下
    ]
    on = [0] * len(segments)
 
    # 循环遍历数码管中的每一段
    for (i, ((xA, yA), (xB, yB))) in enumerate(segments): # 检测分割后的ROI区域，并统计分割图中的阈值像素点
        segROI = roi[yA:yB, xA:xB]
        total = cv2.countNonZero(segROI)
        area = (xB - xA) * (yB - yA)
 
        # 如果非零区域的个数大于整个区域的一半，则认为该段是亮的
        if total / float(area) > 0.5:
            on[i]= 1
 
    # 进行数字查询并显示结果
    print(tuple(on))
    digit = DIGITS_LOOKUP[tuple(on)]
    digits.append(digit)
    cv2.rectangle(output, (x, y), (x + w, y + h), (0, 255, 0), 1)
    cv2.putText(output, str(digit), (x - 10, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.65, (0, 255, 0), 2)
 
# 显示最终的输出结果
print(u"{}{}.{} \u00b0C".format(*digits))
cv2.imshow("Input", image)
cv2.imshow("Output", output)
cv2.waitKey(0)

(1, 1, 1, 1, 0, 1, 1)
(0, 1, 0, 1, 0, 0, 0)


KeyError: (0, 1, 0, 1, 0, 0, 0)

In [21]:
# -*- coding:utf-8 -*-

import cv2
import numpy as np
import matplotlib.pyplot as plt
import argparse

DIGITS_LOOKUP = {
    (1, 1, 1, 1, 1, 1, 0): 0,
    (1, 1, 0, 0, 0, 0, 0): 1,
    (1, 0, 1, 1, 0, 1, 1): 2,
    (1, 1, 1, 0, 0, 1, 1): 3,
    (1, 1, 0, 0, 1, 0, 1): 4,
    (0, 1, 1, 0, 1, 1, 1): 5,
    (0, 1, 1, 1, 1, 1, 1): 6,
    (1, 1, 0, 0, 0, 1, 0): 7,
    (1, 1, 1, 1, 1, 1, 1): 8,
    (1, 1, 1, 0, 1, 1, 1): 9,
    (0, 0, 0, 0, 0, 1, 1): '-'
}
H_W_Ratio = 1.9
THRESHOLD = 35
arc_tan_theta = 6.0  # 数码管倾斜角度
crop_y0 = 215
crop_y1 = 470
crop_x0 = 260
crop_x1 = 890

parser = argparse.ArgumentParser()
parser.add_argument('image_path', help='path to image')
parser.add_argument('-s', '--show_image', action='store_const', const=True, help='whether to show image')
parser.add_argument('-d', '--is_debug', action='store_const', const=True, help='True or False')


def load_image(path, show=False):
    # todo: crop image and clear dc and ac signal
    gray_img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    h, w = gray_img.shape
    # crop_y0 = 0 if h <= crop_y0_init else crop_y0_init
    # crop_y1 = h if h <= crop_y1_init else crop_y1_init
    # crop_x0 = 0 if w <= crop_x0_init else crop_x0_init
    # crop_x1 = w if w <= crop_x1_init else crop_x1_init
    # gray_img = gray_img[crop_y0:crop_y1, crop_x0:crop_x1]
    blurred = cv2.GaussianBlur(gray_img, (7, 7), 0)
    if show:
        cv2.imshow('gray_img', gray_img)
        cv2.imshow('blurred_img', blurred)
    return blurred, gray_img


def preprocess(img, threshold, show=False, kernel_size=(5, 5)):
    # 直方图局部均衡化
    clahe = cv2.createCLAHE(clipLimit=2, tileGridSize=(6, 6))
    img = clahe.apply(img)
    # 自适应阈值二值化
    dst = cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 127, threshold)
    # 闭运算开运算
    kernel = cv2.getStructuringElement(cv2.MORPH_CROSS, kernel_size)
    dst = cv2.morphologyEx(dst, cv2.MORPH_CLOSE, kernel)
    dst = cv2.morphologyEx(dst, cv2.MORPH_OPEN, kernel)

    if show:
        cv2.imshow('equlizeHist', img)
        cv2.imshow('threshold', dst)
    return dst


def helper_extract(one_d_array, threshold=20):
    res = []
    flag = 0
    temp = 0
    for i in range(len(one_d_array)):
        if one_d_array[i] < 12 * 255:
            if flag > threshold:
                start = i - flag
                end = i
                temp = end
                if end - start > 20:
                    res.append((start, end))
            flag = 0
        else:
            flag += 1

    else:
        if flag > threshold:
            start = temp
            end = len(one_d_array)
            if end - start > 50:
                res.append((start, end))
    return res


def find_digits_positions(img, reserved_threshold=20):
    # cnts = cv2.findContours(img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    # digits_positions = []
    # for c in cnts[1]:
    #     (x, y, w, h) = cv2.boundingRect(c)
    #     cv2.rectangle(img, (x, y), (x + w, y + h), (128, 0, 0), 2)
    #     cv2.imshow('test', img)
    #     cv2.waitKey(0)
    #     cv2.destroyWindow('test')
    #     if w >= reserved_threshold and h >= reserved_threshold:
    #         digit_cnts.append(c)
    # if digit_cnts:
    #     digit_cnts = contours.sort_contours(digit_cnts)[0]

    digits_positions = []
    img_array = np.sum(img, axis=0)
    horizon_position = helper_extract(img_array, threshold=reserved_threshold)
    img_array = np.sum(img, axis=1)
    vertical_position = helper_extract(img_array, threshold=reserved_threshold * 4)
    # make vertical_position has only one element
    if len(vertical_position) > 1:
        vertical_position = [(vertical_position[0][0], vertical_position[len(vertical_position) - 1][1])]
    for h in horizon_position:
        for v in vertical_position:
            digits_positions.append(list(zip(h, v)))
    assert len(digits_positions) > 0, "Failed to find digits's positions"

    return digits_positions


def recognize_digits_area_method(digits_positions, output_img, input_img):
    digits = []
    for c in digits_positions:
        x0, y0 = c[0]
        x1, y1 = c[1]
        roi = input_img[y0:y1, x0:x1]
        h, w = roi.shape
        suppose_W = max(1, int(h / H_W_Ratio))
        # 对1的情况单独识别
        if w < suppose_W / 2:
            x0 = x0 + w - suppose_W
            w = suppose_W
            roi = input_img[y0:y1, x0:x1]
        width = (max(int(w * 0.15), 1) + max(int(h * 0.15), 1)) // 2
        dhc = int(width * 0.8)
        # print('width :', width)
        # print('dhc :', dhc)

        small_delta = int(h / arc_tan_theta) // 4
        # print('small_delta : ', small_delta)
        segments = [
            # # version 1
            # ((w - width, width // 2), (w, (h - dhc) // 2)),
            # ((w - width - small_delta, (h + dhc) // 2), (w - small_delta, h - width // 2)),
            # ((width // 2, h - width), (w - width // 2, h)),
            # ((0, (h + dhc) // 2), (width, h - width // 2)),
            # ((small_delta, width // 2), (small_delta + width, (h - dhc) // 2)),
            # ((small_delta, 0), (w, width)),
            # ((width, (h - dhc) // 2), (w - width, (h + dhc) // 2))

            # # version 2
            ((w - width - small_delta, width // 2), (w, (h - dhc) // 2)),
            ((w - width - 2 * small_delta, (h + dhc) // 2), (w - small_delta, h - width // 2)),
            ((width - small_delta, h - width), (w - width - small_delta, h)),
            ((0, (h + dhc) // 2), (width, h - width // 2)),
            ((small_delta, width // 2), (small_delta + width, (h - dhc) // 2)),
            ((small_delta, 0), (w + small_delta, width)),
            ((width - small_delta, (h - dhc) // 2), (w - width - small_delta, (h + dhc) // 2))
        ]
        # cv2.rectangle(roi, segments[0][0], segments[0][1], (128, 0, 0), 2)
        # cv2.rectangle(roi, segments[1][0], segments[1][1], (128, 0, 0), 2)
        # cv2.rectangle(roi, segments[2][0], segments[2][1], (128, 0, 0), 2)
        # cv2.rectangle(roi, segments[3][0], segments[3][1], (128, 0, 0), 2)
        # cv2.rectangle(roi, segments[4][0], segments[4][1], (128, 0, 0), 2)
        # cv2.rectangle(roi, segments[5][0], segments[5][1], (128, 0, 0), 2)
        # cv2.rectangle(roi, segments[6][0], segments[6][1], (128, 0, 0), 2)
        # cv2.imshow('i', roi)
        # cv2.waitKey()
        # cv2.destroyWindow('i')
        on = [0] * len(segments)

        for (i, ((xa, ya), (xb, yb))) in enumerate(segments):
            seg_roi = roi[ya:yb, xa:xb]
            # plt.imshow(seg_roi)
            # plt.show()
            total = cv2.countNonZero(seg_roi)
            area = (xb - xa) * (yb - ya) * 0.9
            print(total / float(area))
            if total / float(area) > 0.45:
                on[i] = 1

        # print(on)

        if tuple(on) in DIGITS_LOOKUP.keys():
            digit = DIGITS_LOOKUP[tuple(on)]
        else:
            digit = '*'
        digits.append(digit)
        cv2.rectangle(output_img, (x0, y0), (x1, y1), (0, 128, 0), 2)
        cv2.putText(output_img, str(digit), (x0 - 10, y0 + 10), cv2.FONT_HERSHEY_SIMPLEX, 0.65, (0, 128, 0), 2)

    return digits


def recognize_digits_line_method(digits_positions, output_img, input_img):
    digits = []
    for c in digits_positions:
        x0, y0 = c[0]
        x1, y1 = c[1]
        roi = input_img[y0:y1, x0:x1]
        h, w = roi.shape
        suppose_W = max(1, int(h / H_W_Ratio))

        # 消除无关符号干扰
        if x1 - x0 < 25 and cv2.countNonZero(roi) / ((y1 - y0) * (x1 - x0)) < 0.2:
            continue

        # 对1的情况单独识别
        if w < suppose_W / 2:
            x0 = max(x0 + w - suppose_W, 0)
            roi = input_img[y0:y1, x0:x1]
            w = roi.shape[1]

        center_y = h // 2
        quater_y_1 = h // 4
        quater_y_3 = quater_y_1 * 3
        center_x = w // 2
        line_width = 5  # line's width
        width = (max(int(w * 0.15), 1) + max(int(h * 0.15), 1)) // 2
        small_delta = int(h / arc_tan_theta) // 4
        segments = [
            ((w - 2 * width, quater_y_1 - line_width), (w, quater_y_1 + line_width)),
            ((w - 2 * width, quater_y_3 - line_width), (w, quater_y_3 + line_width)),
            ((center_x - line_width - small_delta, h - 2 * width), (center_x - small_delta + line_width, h)),
            ((0, quater_y_3 - line_width), (2 * width, quater_y_3 + line_width)),
            ((0, quater_y_1 - line_width), (2 * width, quater_y_1 + line_width)),
            ((center_x - line_width, 0), (center_x + line_width, 2 * width)),
            ((center_x - line_width, center_y - line_width), (center_x + line_width, center_y + line_width)),
        ]
        on = [0] * len(segments)

        for (i, ((xa, ya), (xb, yb))) in enumerate(segments):
            seg_roi = roi[ya:yb, xa:xb]
            # plt.imshow(seg_roi, 'gray')
            # plt.show()
            total = cv2.countNonZero(seg_roi)
            area = (xb - xa) * (yb - ya) * 0.9
            # print('prob: ', total / float(area))
            if total / float(area) > 0.25:
                on[i] = 1
        # print('encode: ', on)
        if tuple(on) in DIGITS_LOOKUP.keys():
            digit = DIGITS_LOOKUP[tuple(on)]
        else:
            digit = '*'

        digits.append(digit)

        # 小数点的识别
        # print('dot signal: ',cv2.countNonZero(roi[h - int(3 * width / 4):h, w - int(3 * width / 4):w]) / (9 / 16 * width * width))
        if cv2.countNonZero(roi[h - int(3 * width / 4):h, w - int(3 * width / 4):w]) / (9. / 16 * width * width) > 0.65:
            digits.append('.')
            cv2.rectangle(output_img,
                          (x0 + w - int(3 * width / 4), y0 + h - int(3 * width / 4)),
                          (x1, y1), (0, 128, 0), 2)
            cv2.putText(output_img, 'dot',
                        (x0 + w - int(3 * width / 4), y0 + h - int(3 * width / 4) - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.65, (0, 128, 0), 2)

        cv2.rectangle(output_img, (x0, y0), (x1, y1), (0, 128, 0), 2)
        cv2.putText(output_img, str(digit), (x0 + 3, y0 + 10), cv2.FONT_HERSHEY_SIMPLEX, 0.65, (0, 128, 0), 2)
    return digits


def main():
    args = parser.parse_args()
    blurred, gray_img = load_image(args.image_path, show=args.show_image)
    output = blurred
    dst = preprocess(blurred, THRESHOLD, show=args.show_image)
    digits_positions = find_digits_positions(dst)
    digits = recognize_digits_line_method(digits_positions, output, dst)
    if args.show_image:
        cv2.imshow('output', output)
        cv2.waitKey()
        cv2.destroyAllWindows()
    print(digits)


if __name__ == '__main__':
    main()

usage: ipykernel_launcher.py [-h] [-s] [-d] image_path
ipykernel_launcher.py: error: unrecognized arguments: -f


SystemExit: 2

F:\ProgramData\Anaconda3\envs\Deep_learn\lib\site-packages\IPython\core\interactiveshell.py:3405: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [22]:
import numpy as np
import matplotlib.pyplot as plt
import cv2

# Read Image
BGR = cv2.imread('sample.jpg')
RGB = cv2.cvtColor(BGR, cv2.COLOR_BGR2RGB)

# Channels split
R = BGR[...,2]
G = BGR[...,1]
B = BGR[...,0]

# Threshold per channel
R[B>120] = 0
R[G>120] = 0
R[R<230] = 0

# Binarize
Binary = cv2.threshold(R, 127, 255, cv2.THRESH_BINARY)[1]
# Edge Detection
Edges = cv2.Canny(Binary, 50, 200)

# Read Template
templBGR = cv2.imread('templ.png')
templRGB =  cv2.cvtColor(templBGR, cv2.COLOR_BGR2RGB)
templateGray =  cv2.cvtColor(templBGR, cv2.COLOR_BGR2GRAY)
# Binarize Template
templateBinary = cv2.threshold(templateGray, 84, 255, cv2.THRESH_BINARY)[1]
# Denoise Template
templateFiltered = cv2.medianBlur(templateBinary,7)
# Resize Template
template = cv2.resize(templateFiltered, (templBGR.shape[1]//2, templBGR.shape[0]//2))
# Edge Detection Template
templateEdges = cv2.Canny(template, 50, 200)
# Extract Dimensions
h, w = template.shape

res = cv2.matchTemplate(Edges,templateEdges,cv2.TM_CCORR)

(_, _, _, maxLoc) = cv2.minMaxLoc(res)

img = RGB.copy()
cv2.rectangle(img, (maxLoc[0], maxLoc[1]), (maxLoc[0] + w, maxLoc[1] + h), (255,255,128), 2)

plt.subplot(221)
plt.imshow(RGB)
plt.title('Original')
plt.axis('off')

plt.subplot(222)
plt.imshow(Edges, cmap='gray')
plt.title('Segmented')
plt.axis('off')

plt.subplot(223)
plt.imshow(templRGB)
plt.title('Template')
plt.axis('off')

plt.subplot(224)
plt.imshow(img)
plt.title('Result')
plt.axis('off')

plt.show()

error: OpenCV(4.6.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'
